In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sympy import *


Consecutive blocks if MORE THAN CHANCE
Consecutive blocks if PAID MORE

GAS FEE VS. MEV FEE??


Questions:
What can be done with Multi-block MEV? Taxonomy
Q2: How likely, often?
Q3: How cheap is a relay block?
Q6: Profit of MEV blocks over time and count

### cleaning

In [ ]:
df_b = pd.read_csv('data/BBRs_5538500_5538999.csv')


In [2]:
df_p = pd.read_csv('data/PPDs_5538500_5540000.csv')
df_p

,slot,block_hash,block_reward,gas_used,validator_pubkey,operator_name,pool_name,buidler_pubkey,builder_name,relay_name,relay_url
0,5538500,0x9655d36ce0e876eea03fde593d875e3fed71bf138af6...,0.159454,13732781,0x92b4ed3d1682d2586e68878d6d0629e555d29eb766ed...,NaN,NaN,0x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c...,Unknown,Flashbots,https://0xac6e77dfe25ecd6110b8e780608cce0dab71...
1,5538501,0x383dca1fb965bcb19f92932782e0efbbe3f8d8bc15b8...,0.056432,14759390,0x98226d1be207533941452ecf37d09adaf234bf4d3163...,NaN,Coinbase,0xa1daf0ab37a9a204bc5925717f78a795fa2812f8fba8...,Ø gethgo1.19.1 linux,Flashbots,https://0xac6e77dfe25ecd6110b8e780608cce0dab71...
2,5538502,0x37159c71a33375586597bd896392ceb6317bd385a97a...,0.136855,15351496,0x81509611e5b714536d85ee906bc9d06a8461c8c3dd61...,Stakefish,NaN,0xaec4ec48c2ec03c418c599622980184e926f0de3c9ce...,beaverbuild.org,Bloxroute Regulated,https://0xb0b07cd0abef743db4260b0ed50619cf6ad4...
3,5538503,0x5342aabc7ac2ab398139d4989ef4ff57938d30536ff7...,0.023790,14418555,0xb474cb46ea2bae46ee9e658683662db5f365ec8ddcac...,NaN,Bitfinex,0x81babeec8c9f2bb9c329fd8a3b176032fe0ab5f3b92a...,flashbots,Flashbots,https://0xac6e77dfe25ecd6110b8e780608cce0dab71...
4,5538504,0x498f3ec769d9be888b4f9f23a453a1579442de210abe...,0.052001,18232404,0xa6b6364d808331a54fd3ec24f7b0ac2d1e43e884e523...,NaN,NaN,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,flashbots,Flashbots,https://0xac6e77dfe25ecd6110b8e780608cce0dab71...
...,...,...,...,...,...,...,...,...,...,...,...
1368,5539995,0xe40fa3e78963d61fb42f6e341911f92dfd73b3544663...,0.034382,11904764,0x8acfb26ffe42d89c1a536c1eaf756858e38447ff680b...,NaN,NaN,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,flashbots,Flashbots,https://0xac6e77dfe25ecd6110b8e780608cce0dab71...
1369,5539996,0x790a324e2793e8ea44ec3629d3d5a3faa9ef1b8dc5be...,0.229077,15792512,0xa59bcaaa071bdbdbb7e64201e7383efc150515036c56...,NaN,Kraken,0xa4fb63c2ceeee73d1f1711fadf1c5357ac98cecb999d...,builder0x69,Flashbots,https://0xac6e77dfe25ecd6110b8e780608cce0dab71...
1370,5539998,0xb2577d212891dd3da5b21a58e40c40a3b8e43967a4ba...,0.071430,17799874,0xb143f8647d8fe34e82537c243fe9670fa91257b6c4ca...,NaN,NaN,0x94aa4ee318f39b56547a253700917982f4b737a49fc3...,bloxroute,Bloxroute Max Profit,https://0x8b5d2e73e2a3a55c6c87b8b6eb92e0149a12...
1371,5539999,0x8d86751ae37b5750a1481171cb386202b612396eef2e...,0.039707,12504193,0xaa080e8c227f979b139633bd7a874a6497656042a034...,NaN,Kraken,0x83bee51799e65130296f5d60ce7d78119578db6045c3...,Unknown,Flashbots,https://0xac6e77dfe25ecd6110b8e780608cce0dab71...


In [3]:
df_p.drop_duplicates(subset=['slot'], keep='first', inplace=True)

In [4]:
df_p.sort_values(by=['slot'], ascending=True, inplace=True)
df_p.rename(columns={'buidler_pubkey': 'builder_pubkey'}, inplace=True)


In [5]:
df_p = df_p[['slot', 'block_hash', 'block_reward', 'gas_used', 'validator_pubkey', 'builder_pubkey', 'builder_name', 'relay_name']]


In [6]:
df_p['boosted'] = True

/var/folders/1l/dlmzk3fn30g9wwk6y58j36c00000gn/T/ipykernel_52674/3279590111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p['boosted'] = True


In [7]:
slot = np.arange(5538500,5540000)
df = pd.DataFrame(slot, columns = ['slot'])


In [8]:
df = df.merge(df_p, how='left', on='slot')
df['builder_name'].fillna('No builder', inplace=True)
df['builder_name'].replace('Ø\x83\x01\x0b\x84geth\x88go1.19.1\x85linux', 'coda_geth_linux', inplace = True)


,slot,block_hash,block_reward,gas_used,validator_pubkey,builder_pubkey,builder_name,relay_name,boosted
0,5538500,0x9655d36ce0e876eea03fde593d875e3fed71bf138af6...,0.159454,13732781.0,0x92b4ed3d1682d2586e68878d6d0629e555d29eb766ed...,0x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c...,Unknown,Flashbots,True
1,5538501,0x383dca1fb965bcb19f92932782e0efbbe3f8d8bc15b8...,0.056432,14759390.0,0x98226d1be207533941452ecf37d09adaf234bf4d3163...,0xa1daf0ab37a9a204bc5925717f78a795fa2812f8fba8...,coda_geth_linux,Flashbots,True
2,5538502,0x37159c71a33375586597bd896392ceb6317bd385a97a...,0.136855,15351496.0,0x81509611e5b714536d85ee906bc9d06a8461c8c3dd61...,0xaec4ec48c2ec03c418c599622980184e926f0de3c9ce...,beaverbuild.org,Bloxroute Regulated,True
3,5538503,0x5342aabc7ac2ab398139d4989ef4ff57938d30536ff7...,0.023790,14418555.0,0xb474cb46ea2bae46ee9e658683662db5f365ec8ddcac...,0x81babeec8c9f2bb9c329fd8a3b176032fe0ab5f3b92a...,flashbots,Flashbots,True
4,5538504,0x498f3ec769d9be888b4f9f23a453a1579442de210abe...,0.052001,18232404.0,0xa6b6364d808331a54fd3ec24f7b0ac2d1e43e884e523...,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,flashbots,Flashbots,True
...,...,...,...,...,...,...,...,...,...
1495,5539995,0xe40fa3e78963d61fb42f6e341911f92dfd73b3544663...,0.034382,11904764.0,0x8acfb26ffe42d89c1a536c1eaf756858e38447ff680b...,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,flashbots,Flashbots,True
1496,5539996,0x790a324e2793e8ea44ec3629d3d5a3faa9ef1b8dc5be...,0.229077,15792512.0,0xa59bcaaa071bdbdbb7e64201e7383efc150515036c56...,0xa4fb63c2ceeee73d1f1711fadf1c5357ac98cecb999d...,builder0x69,Flashbots,True
1497,5539997,NaN,NaN,NaN,NaN,NaN,No builder,NaN,NaN
1498,5539998,0xb2577d212891dd3da5b21a58e40c40a3b8e43967a4ba...,0.071430,17799874.0,0xb143f8647d8fe34e82537c243fe9670fa91257b6c4ca...,0x94aa4ee318f39b56547a253700917982f4b737a49fc3...,bloxroute,Bloxroute Max Profit,True


### descriptive statistics

ToDo: a) take a look at Relayer

In [10]:
fltr = 'builder_name'
#fltr = 'builder_pubkey'

df['count'] = 1
df['position'] = 1
count = 0
for i in range(0, len(df)):
    #check = df['builder_pubkey'].iloc[i] == df['builder_pubkey'].iloc[i-1] 
    check = (df[fltr].iloc[i] == df[fltr].iloc[i-1]) & (df[fltr].iloc[i] != 'Unknown')
    
    if check == True:
        if count == 0:
            count = 2
        else:
            count += 1

    elif (check == False) & (count == 0):
            continue

    elif (check == False) & (count != 0):
        for j in range(1, count+1):
            df['count'].iloc[i-j] = count
            df['position'].iloc[i-j] = count+1-j

        count = 0




/var/folders/1l/dlmzk3fn30g9wwk6y58j36c00000gn/T/ipykernel_52674/1070089808.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count'].iloc[i-j] = count
/var/folders/1l/dlmzk3fn30g9wwk6y58j36c00000gn/T/ipykernel_52674/1070089808.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['position'].iloc[i-j] = count+1-j


In [11]:
df['block_reward'].fillna((df[df['count'] == 1]['block_reward'].mean()), inplace=True)


df_backup = df
df

,slot,block_hash,block_reward,gas_used,validator_pubkey,builder_pubkey,builder_name,relay_name,boosted,count,position
0,5538500,0x9655d36ce0e876eea03fde593d875e3fed71bf138af6...,0.159454,13732781.0,0x92b4ed3d1682d2586e68878d6d0629e555d29eb766ed...,0x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c...,Unknown,Flashbots,True,1,1
1,5538501,0x383dca1fb965bcb19f92932782e0efbbe3f8d8bc15b8...,0.056432,14759390.0,0x98226d1be207533941452ecf37d09adaf234bf4d3163...,0xa1daf0ab37a9a204bc5925717f78a795fa2812f8fba8...,coda_geth_linux,Flashbots,True,1,1
2,5538502,0x37159c71a33375586597bd896392ceb6317bd385a97a...,0.136855,15351496.0,0x81509611e5b714536d85ee906bc9d06a8461c8c3dd61...,0xaec4ec48c2ec03c418c599622980184e926f0de3c9ce...,beaverbuild.org,Bloxroute Regulated,True,1,1
3,5538503,0x5342aabc7ac2ab398139d4989ef4ff57938d30536ff7...,0.023790,14418555.0,0xb474cb46ea2bae46ee9e658683662db5f365ec8ddcac...,0x81babeec8c9f2bb9c329fd8a3b176032fe0ab5f3b92a...,flashbots,Flashbots,True,2,1
4,5538504,0x498f3ec769d9be888b4f9f23a453a1579442de210abe...,0.052001,18232404.0,0xa6b6364d808331a54fd3ec24f7b0ac2d1e43e884e523...,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,flashbots,Flashbots,True,2,2
...,...,...,...,...,...,...,...,...,...,...,...
1495,5539995,0xe40fa3e78963d61fb42f6e341911f92dfd73b3544663...,0.034382,11904764.0,0x8acfb26ffe42d89c1a536c1eaf756858e38447ff680b...,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,flashbots,Flashbots,True,1,1
1496,5539996,0x790a324e2793e8ea44ec3629d3d5a3faa9ef1b8dc5be...,0.229077,15792512.0,0xa59bcaaa071bdbdbb7e64201e7383efc150515036c56...,0xa4fb63c2ceeee73d1f1711fadf1c5357ac98cecb999d...,builder0x69,Flashbots,True,1,1
1497,5539997,NaN,0.094004,NaN,NaN,NaN,No builder,NaN,NaN,1,1
1498,5539998,0xb2577d212891dd3da5b21a58e40c40a3b8e43967a4ba...,0.071430,17799874.0,0xb143f8647d8fe34e82537c243fe9670fa91257b6c4ca...,0x94aa4ee318f39b56547a253700917982f4b737a49fc3...,bloxroute,Bloxroute Max Profit,True,1,1


In [ ]:
print(len(df[df['boosted'] != True]), 'blocks without PBS') 
print(len(df[df['boosted'] == True]), 'blocks with PBS')
print(len(df[df['boosted'] == True])/len(df)*100, '%')


In [ ]:
stat = df.groupby(df['count']).size().values

for i, x in enumerate(df['count'].unique()):
    stat[i] = stat[i]/x

print('In a dataset of', len(df), 'blocks', int(len(df)*12/60/60), 'hours we find:')

for i, x in enumerate(stat):
    print(x, 'instances of', i+1, 'consecutive blocks')

print('proposed by the same builder')

In [72]:
print(df['builder_name'].unique())
print(df.groupby(df['builder_name']).size().sort_values(ascending=False))

['Unknown' 'coda_geth_linux' 'beaverbuild.org' 'flashbots' 'No builder'
 'builder0x69' 'bloxroute' 'blocknative' 'eden' 'https://eth-builder.com']
builder_name
beaverbuild.org            392
flashbots                  347
bloxroute                  186
No builder                 165
builder0x69                155
Unknown                    146
coda_geth_linux             48
blocknative                 47
https://eth-builder.com      9
eden                         5
dtype: int64


In [73]:
temp = df[df['count'] >= 2]
temp.groupby(['builder_name', 'count']).size()

builder_name     count
No builder       2         34
                 3          6
beaverbuild.org  2        100
                 3         39
                 4         16
                 5         10
blocknative      2          2
bloxroute        2         42
                 3          9
builder0x69      2         24
                 3         12
coda_geth_linux  2          2
flashbots        2         96
                 3         24
                 4          4
dtype: int64

### plotting descriptive

In [ ]:
bins = np.arange(df['slot'].min(), df['slot'].max(), 5*60)
data = []
for i in range(0,5):
        data.append(df[(df['slot'] >= bins[i]) & (df['slot'] < bins[i] + 5*60)]['block_reward'].mean())

In [ ]:
fig, ax = plt.subplots(figsize =(8, 6))

plt.bar(['1h','2h','3h','4h','5h'], data)

ax.set_ylabel('Builder payment')
ax.set_xlabel('Slot')

vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2f}'.format(x)+' ETH' for x in vals])

# show plot
plt.show()

### plotting analysis

In [ ]:
data = []
for x in df['count'].unique():
    data.append(list(df[df['count']==x]['block_reward']))

fig, ax = plt.subplots(figsize =(8, 8))

ax.boxplot(data, showfliers=False)

ax.set_ylabel('Builder payment')
ax.set_xlabel('Consecutive blocks')

vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2f}'.format(x)+' ETH' for x in vals])

# show plot
plt.show()


In [ ]:
def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

In [ ]:
#how much do builder pay on average for consecutive blocks

fig, ax = plt.subplots(figsize =(8, 8))

x = list(df['position'].unique())

ax.plot(x, df['block_reward'].groupby(df['position']).median())

y1 = df.groupby(['position']).agg({'block_reward': [q25]})
y2 = df.groupby(['position']).agg({'block_reward': [q75]})
ax.plot(x, y1, x, y2, color='grey')

ax.fill_between(x, list(y1.iloc[:, 0]), list(y2.iloc[:, 0]), facecolor='lightgrey', interpolate=True)


data = df[df['count'] >= 2]
ax.scatter(data['position'], data['block_reward'])

ax.set_ylabel('Mean builder payment')
ax.set_xlabel('n position in consecutive blocks')

plt.ylim(0, .5)

vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2f}'.format(x)+' ETH' for x in vals])



plt.xticks(list(df['position'].unique()))
#plt.xticks(list(df['position'].unique())[1:])


# show plot
plt.show()


In [ ]:
# Is there a pattern, to propose low gas blocks?
# "VALIDATOR for slot k+1 is buying the slot in order to submit an empty block just before the slot they are elected for"
# > Validators could be "unknown"

n, bins, patches = plt.hist(df[df['boosted']==True]['gas_used'], 50, density=True, facecolor='g', alpha=0.75)

df[(df['boosted']==True) & (df['gas_used'] < 0.8e7)]

### math

In [148]:
df

,slot,block_hash,block_reward,gas_used,validator_pubkey,builder_pubkey,builder_name,relay_name,boosted,count,position
0,5538500,0x9655d36ce0e876eea03fde593d875e3fed71bf138af6...,0.159454,13732781.0,0x92b4ed3d1682d2586e68878d6d0629e555d29eb766ed...,0x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c...,Unknown,Flashbots,True,1,1
1,5538501,0x383dca1fb965bcb19f92932782e0efbbe3f8d8bc15b8...,0.056432,14759390.0,0x98226d1be207533941452ecf37d09adaf234bf4d3163...,0xa1daf0ab37a9a204bc5925717f78a795fa2812f8fba8...,coda_geth_linux,Flashbots,True,1,1
2,5538502,0x37159c71a33375586597bd896392ceb6317bd385a97a...,0.136855,15351496.0,0x81509611e5b714536d85ee906bc9d06a8461c8c3dd61...,0xaec4ec48c2ec03c418c599622980184e926f0de3c9ce...,beaverbuild.org,Bloxroute Regulated,True,1,1
3,5538503,0x5342aabc7ac2ab398139d4989ef4ff57938d30536ff7...,0.023790,14418555.0,0xb474cb46ea2bae46ee9e658683662db5f365ec8ddcac...,0x81babeec8c9f2bb9c329fd8a3b176032fe0ab5f3b92a...,flashbots,Flashbots,True,2,1
4,5538504,0x498f3ec769d9be888b4f9f23a453a1579442de210abe...,0.052001,18232404.0,0xa6b6364d808331a54fd3ec24f7b0ac2d1e43e884e523...,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,flashbots,Flashbots,True,2,2
...,...,...,...,...,...,...,...,...,...,...,...
1495,5539995,0xe40fa3e78963d61fb42f6e341911f92dfd73b3544663...,0.034382,11904764.0,0x8acfb26ffe42d89c1a536c1eaf756858e38447ff680b...,0x81beef03aafd3dd33ffd7deb337407142c80fea2690e...,flashbots,Flashbots,True,1,1
1496,5539996,0x790a324e2793e8ea44ec3629d3d5a3faa9ef1b8dc5be...,0.229077,15792512.0,0xa59bcaaa071bdbdbb7e64201e7383efc150515036c56...,0xa4fb63c2ceeee73d1f1711fadf1c5357ac98cecb999d...,builder0x69,Flashbots,True,1,1
1497,5539997,NaN,0.094004,NaN,NaN,NaN,No builder,NaN,NaN,1,1
1498,5539998,0xb2577d212891dd3da5b21a58e40c40a3b8e43967a4ba...,0.071430,17799874.0,0xb143f8647d8fe34e82537c243fe9670fa91257b6c4ca...,0x94aa4ee318f39b56547a253700917982f4b737a49fc3...,bloxroute,Bloxroute Max Profit,True,1,1


In [27]:
def prob(c, p):
    # c = consecutive blocks
    # n = slots
    # p = share MEV boost
    
    x = symbols('x', real=True)

    c = c; n = 32; p = p

    xx = min(i for i in solve(1-x+(1-p)*p**c*x**(c+1)) if i > 1)
    qn = ((1 - p*xx) / ((c + 1 - c*xx)*(1-p))) * (1/(xx**(n+1)))
    
    return qn*100, (1-qn)*100

    

In [246]:
lst = list(df['builder_name'].unique())
calc = pd.DataFrame(lst, columns = ['builder_name'])


In [247]:
temp_lst = []
for x in lst:
    temp_lst.append(len(df[df['builder_name'] == x]))

calc['slots'] = temp_lst

calc['share'] = np.round(calc['slots']/len(df),4)

for i in range(1,8+1):
    temp_lst = []
    
    for x in lst:
        temp_lst.append(int(len(df[(df['builder_name'] == x) & (temp['count'] == i)])/i))
    
    name = f'k_{i}'
    calc[name] = temp_lst
    

In [248]:
calc

,builder_name,slots,share,k_1,k_2,k_3,k_4,k_5,k_6,k_7,k_8
0,Unknown,146,0.0973,146,0,0,0,0,0,0,0
1,coda_geth_linux,48,0.0320,46,1,0,0,0,0,0,0
2,beaverbuild.org,392,0.2613,227,50,13,4,2,0,0,0
3,flashbots,347,0.2313,223,48,8,1,0,0,0,0
4,No builder,165,0.1100,125,17,2,0,0,0,0,0
5,builder0x69,155,0.1033,119,12,4,0,0,0,0,0
6,bloxroute,186,0.1240,135,21,3,0,0,0,0,0
7,blocknative,47,0.0313,45,1,0,0,0,0,0,0
8,eden,5,0.0033,5,0,0,0,0,0,0,0
9,https://eth-builder.com,9,0.0060,9,0,0,0,0,0,0,0


In [202]:
#divide by i to count instances
df.groupby(['builder_name', 'count']).size()

builder_name             count
No builder               1        125
                         2         34
                         3          6
Unknown                  1        146
beaverbuild.org          1        227
                         2        100
                         3         39
                         4         16
                         5         10
blocknative              1         45
                         2          2
bloxroute                1        135
                         2         42
                         3          9
builder0x69              1        119
                         2         24
                         3         12
coda_geth_linux          1         46
                         2          2
eden                     1          5
flashbots                1        223
                         2         96
                         3         24
                         4          4
https://eth-builder.com  1          9
dtype: int64

In [260]:
lst = list(df['builder_name'].unique())
expt = calc[['builder_name', 'slots', 'share']]


In [267]:
epochs = np.round(len(df)/32, 0)

47.0

In [261]:
for c in range(2,8+1):
    temp_lst = []
    
    for x in lst:
        p = float(new[new['builder_name'] == x]['share'].values)
        no_success, success = prob(c, p)
        temp_lst.append(int(np.round(float(success)/100*epochs,0)))
    
    name = f'e_{c}'
    expt[name] = temp_lst
    

/var/folders/1l/dlmzk3fn30g9wwk6y58j36c00000gn/T/ipykernel_52674/1207199651.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expt[name] = temp_lst
/var/folders/1l/dlmzk3fn30g9wwk6y58j36c00000gn/T/ipykernel_52674/1207199651.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expt[name] = temp_lst
/var/folders/1l/dlmzk3fn30g9wwk6y58j36c00000gn/T/ipykernel_52674/1207199651.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [263]:
expt

,builder_name,slots,share,e_2,e_3,e_4,e_5,e_6,e_7,e_8
0,Unknown,146,0.0973,11,1,0,0,0,0,0
1,coda_geth_linux,48,0.0320,1,0,0,0,0,0,0
2,beaverbuild.org,392,0.2613,39,16,5,1,0,0,0
3,flashbots,347,0.2313,36,12,3,1,0,0,0
4,No builder,165,0.1100,14,2,0,0,0,0,0
5,builder0x69,155,0.1033,12,1,0,0,0,0,0
6,bloxroute,186,0.1240,17,2,0,0,0,0,0
7,blocknative,47,0.0313,1,0,0,0,0,0,0
8,eden,5,0.0033,0,0,0,0,0,47,47
9,https://eth-builder.com,9,0.0060,0,0,0,0,0,0,47


In [264]:
calc

,builder_name,slots,share,k_1,k_2,k_3,k_4,k_5,k_6,k_7,k_8
0,Unknown,146,0.0973,146,0,0,0,0,0,0,0
1,coda_geth_linux,48,0.0320,46,1,0,0,0,0,0,0
2,beaverbuild.org,392,0.2613,227,50,13,4,2,0,0,0
3,flashbots,347,0.2313,223,48,8,1,0,0,0,0
4,No builder,165,0.1100,125,17,2,0,0,0,0,0
5,builder0x69,155,0.1033,119,12,4,0,0,0,0,0
6,bloxroute,186,0.1240,135,21,3,0,0,0,0,0
7,blocknative,47,0.0313,45,1,0,0,0,0,0,0
8,eden,5,0.0033,5,0,0,0,0,0,0,0
9,https://eth-builder.com,9,0.0060,9,0,0,0,0,0,0,0


In [84]:
p = np.round(len(df[df['builder_name'] == 'beaverbuild.org'])/1500,4)

print('In a dataset of', len(df), 'blocks', int(len(df)*12/60/60), 'hours we expect ..')
for i in [1,2,3,4,5]:
    c = i
    no_success, success = prob(c, p)
    print(i, 'consecutive blocks: Probability', np.round(float(success),2), '% - Instances:', np.round(float(success)/100*47,0))

print('.. proposed by the same builder')


In a dataset of 1500 blocks 5 hours we expect ..
1 consecutive blocks: Probability 99.99 % - Instances: 47.0
2 consecutive blocks: Probability 83.78 % - Instances: 39.0
3 consecutive blocks: Probability 34.21 % - Instances: 16.0
4 consecutive blocks: Probability 9.75 % - Instances: 5.0
5 consecutive blocks: Probability 2.53 % - Instances: 1.0
.. proposed by the same builder


In [204]:
stat = df[df['builder_name'] == 'beaverbuild.org'].groupby(['count']).size().values

print('In a dataset of', len(df), 'blocks', int(len(df)*12/60/60), 'hours we find ..')

for i, x in enumerate(np.sort(df[df['builder_name'] == 'beaverbuild.org']['count'].unique())):
    stat[i] = stat[i]/x
    
    print(stat[i], 'instances of', i+1, 'consecutive blocks')
    
print('.. proposed by the same builder')

In a dataset of 1500 blocks 5 hours we find ..
227 instances of 1 consecutive blocks
50 instances of 2 consecutive blocks
13 instances of 3 consecutive blocks
4 instances of 4 consecutive blocks
2 instances of 5 consecutive blocks
.. proposed by the same builder


In [ ]:
## MATRIX
cons_blocks = np.arange(2,5+1) #cons_blocks
prob = np.arange(.1,.9+.05,.05)

In [24]:
lst = []
for p in prob:
    lst_temp = []
    lst_temp.append(x)
    for c in cons_blocks:
        no_success, success = prob(c, p)
        lst_temp.append(success)
    
    for z in range(0, len(lst_temp)):
        try: 
            lst_temp[z] = np.round(float(lst_temp[z]), 2)
        except TypeError:
            lst_temp[z] = 100
        
    lst.append(lst_temp)
        

TypeError: 'float' object is not iterable

In [ ]:
df = pd.DataFrame(lst, columns =['probability', 'k=2', 'k=3', 'k=4', 'k=5', 'k=6', 'k=7', 'k=8', 'k=9', 'k=10'])
df = df.set_index('probability')
df

In [ ]:
print('If the market share of MEV boost is p=0.5, then we derive', (38.96/100)*225, 'k=5 consecutive blocks a day.')
print('If the market share of MEV boost is p=0.5, then we derive', (38.96/100)*6759, 'k=5 consecutive blocks a month.')
